create the results table for the accessibility paper

In [11]:
import geopandas
import r5py
import shapely as shp
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [12]:
perimeters = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    index='id'
)

regions = perimeters[
    perimeters.index.isin([
        'zurich', 'ebc_zrh_v01_ex_zurich', 'zurich_seefeld'
    ])
]

regions

,geometry
id,
zurich,"POLYGON ((2685056.896 1244517.810, 2684280.851..."
ebc_zrh_v01_ex_zurich,"MULTIPOLYGON (((2671482.755 1238555.241, 26714..."
zurich_seefeld,"POLYGON ((2683900.649 1246885.982, 2684120.894..."


In [13]:
accessibility = snman.io.import_geofile_to_gdf(
    os.path.join(outputs_path, 'accessibility_diff.gpkg')
)

accessibility = gpd.sjoin(
    accessibility,
    regions,
    how='left',
    predicate='within'
).rename(columns={'index_right': 'region'})

accessibility

,index,record,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility_before,accessibility_cycling_before,...,accessibility_cycling_before_log,accessibility_cycling_after_log,accessibility_cycling_log_diff_perc,accessibility_foot_diff,accessibility_foot_diff_perc,accessibility_foot_before_log,accessibility_foot_after_log,accessibility_foot_log_diff_perc,geometry,region
0,0,7750768,38,1,1,301,3,68672447.0,69218.961169,50043.444111,...,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978,POINT (2686663.001 1244743.001),ebc_zrh_v01_ex_zurich
1,1,9811619,48,2,2,-2,-2,68672447.0,69218.961169,50043.444111,...,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978,POINT (2686744.001 1244666.001),ebc_zrh_v01_ex_zurich
2,2,11335928,56,1,2,201,2,68672447.0,69218.961169,50043.444111,...,10.820647,11.044736,0.020709,-38.798105,-0.008234,8.457866,8.449598,-0.000978,POINT (2686703.001 1244729.001),ebc_zrh_v01_ex_zurich
3,3,537284,13,1,1,-2,-2,69702513.0,66234.221712,41129.292573,...,10.624476,10.656659,0.003029,0.812481,0.000624,7.171789,7.172413,0.000087,POINT (2697032.001 1251290.001),ebc_zrh_v01_ex_zurich
4,4,3295332,20,2,1,-2,-2,69702513.0,66234.221712,41129.292573,...,10.624476,10.656659,0.003029,0.812481,0.000624,7.171789,7.172413,0.000087,POINT (2696996.001 1251309.001),ebc_zrh_v01_ex_zurich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40690,1179,4492458,30,2,1,-2,-2,68222474.0,93008.739384,100407.037620,...,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391,POINT (2682156.001 1247396.001),zurich
40691,1180,7631781,34,2,2,302,3,68222474.0,93008.739384,100407.037620,...,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391,POINT (2682230.001 1247436.001),zurich
40692,1181,9590269,0,2,1,-2,-2,68222474.0,93008.739384,100407.037620,...,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391,POINT (2682164.001 1247428.001),zurich
40693,1182,11214605,49,1,2,202,2,68222474.0,93008.739384,100407.037620,...,11.516988,11.640142,0.010693,417.121038,0.014398,10.274054,10.288349,0.001391,POINT (2682249.001 1247438.001),zurich


Add group labels

In [15]:
accessibility['x'] = 'all'
accessibility['y'] = 'all'

accessibility['age_group'] = accessibility.apply(
    lambda row: 'age<=25' if row['age'] <= 25 else 'age>=60' if row['age'] >= 60 else 'age:other',
    axis=1
)

accessibility['nationality'] = accessibility.apply(
    lambda row: 'nationality:swiss' if row['residencepermit'] <= -2 else 'nationality:other',
    axis=1
)

accessibility['maritalstatus_type'] = accessibility.apply(
    lambda row: 'maritalstatus:married' if row['maritalstatus'] == 2 else 'maritalstatus:other',
    axis=1
)

accessibility['sex_type'] = accessibility.apply(
    lambda row: 'sex:male' if row['sex'] == 1 else 'sex:female' if row['sex'] == 2 else 'sex:other',
    axis=1
)

stats_values = ['median']

addition = '_log'

fields = {
    'x': ['count'],
    f'accessibility_{MODE_PRIVATE_CARS}_before{addition}': stats_values,
    f'accessibility_{MODE_TRANSIT}_before{addition}': stats_values,
    f'accessibility_{MODE_CYCLING}_before{addition}': stats_values,
    f'accessibility_{MODE_FOOT}_before{addition}': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}_after{addition}': stats_values,
    f'accessibility_{MODE_TRANSIT}_after{addition}': stats_values,
    f'accessibility_{MODE_CYCLING}_after{addition}': stats_values,
    f'accessibility_{MODE_FOOT}_after{addition}': stats_values,
    f'accessibility_{MODE_PRIVATE_CARS}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_TRANSIT}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_CYCLING}{addition}_diff_perc': stats_values,
    f'accessibility_{MODE_FOOT}{addition}_diff_perc': stats_values,
    f'accessibility_before{addition}': stats_values,
    f'accessibility_after{addition}': stats_values,
    f'accessibility{addition}_diff_perc': stats_values,
}

accessibility_stats = {
    'region_and_age': accessibility.groupby(['region', 'age_group']).agg(fields),
    'region_and_nationality': accessibility.groupby(['region', 'nationality']).agg(fields),
    'region_and_sex': accessibility.groupby(['region', 'sex_type']).agg(fields),
    'all': accessibility.groupby(['x', 'y']).agg(fields)
}

res = pd.concat(accessibility_stats.values())
res = res.iloc[::-1]

#for mode in [MODE_CYCLING, MODE_PRIVATE_CARS, MODE_TRANSIT, MODE_FOOT]:
#    res[f'accessibility_{mode}_diff'] = (
#        res[f'accessibility_{mode}_diff'] / res[f'accessibility_{mode}_before']
#    )
    
#res[f'accessibility_diff'] = (
#        res[f'accessibility_diff'] / res[f'accessibility_before']
#    )

res = res.sort_index().transpose()

res.to_excel(
    os.path.join(outputs_path, 'accessibility_results_transposed.xlsx'),
)

res

all  \
                                                          all   
x                                        count   41030.000000   
accessibility_private_cars_before_log    median     11.130990   
accessibility_transit_before_log         median     11.026899   
accessibility_cycling_before_log         median     10.672861   
accessibility_foot_before_log            median      7.985725   
accessibility_private_cars_after_log     median     11.117754   
accessibility_transit_after_log          median     11.026899   
accessibility_cycling_after_log          median     10.712378   
accessibility_foot_after_log             median      7.987779   
accessibility_private_cars_log_diff_perc median     -0.000946   
accessibility_transit_log_diff_perc      median      0.000000   
accessibility_cycling_log_diff_perc      median      0.004147   
accessibility_foot_log_diff_perc         median      0.000045   
accessibility_before_log                 median     11.133993   
accessibility_after_log                  median     11.133876   
accessibility_log_diff_perc              median      0.000596   

                                                ebc_zrh_v01_ex_zurich  \
                                                            age:other   
x                                        count           14769.000000   
accessibility_private_cars_before_log    median             11.113917   
accessibility_transit_before_log         median             10.993044   
accessibility_cycling_before_log         median             10.674313   
accessibility_foot_before_log            median              7.550817   
accessibility_private_cars_after_log     median             11.105319   
accessibility_transit_after_log          median             10.993044   
accessibility_cycling_after_log          median             10.714441   
accessibility_foot_after_log             median              7.551393   
accessibility_private_cars_log_diff_perc median             -0.000415   
accessibility_transit_log_diff_perc      median              0.000000   
accessibility_cycling_log_diff_perc      median              0.003555   
accessibility_foot_log_diff_perc         median              0.000031   
accessibility_before_log                 median             11.117236   
accessibility_after_log                  median             11.119384   
accessibility_log_diff_perc              median              0.000454   

                                                                           \
                                                     age<=25      age>=60   
x                                        count   8120.000000  6870.000000   
accessibility_private_cars_before_log    median    11.113987    11.112450   
accessibility_transit_before_log         median    10.991494    10.990782   
accessibility_cycling_before_log         median    10.670132    10.307474   
accessibility_foot_before_log            median     7.546217     7.500074   
accessibility_private_cars_after_log     median    11.104709    11.103467   
accessibility_transit_after_log          median    10.991494    10.990782   
accessibility_cycling_after_log          median    10.708837    10.333920   
accessibility_foot_after_log             median     7.547063     7.500842   
accessibility_private_cars_log_diff_perc median    -0.000465    -0.000443   
accessibility_transit_log_diff_perc      median     0.000000     0.000000   
accessibility_cycling_log_diff_perc      median     0.003488     0.002466   
accessibility_foot_log_diff_perc         median     0.000030     0.000028   
accessibility_before_log                 median    11.116187    11.110066   
accessibility_after_log                  median    11.118289    11.109206   
accessibility_log_diff_perc              median     0.000393    -0.000001   

                                                                   \
                                                nationality:other   
x                                        count 